In [8]:
# Aidan Fischer
# CS584 HW1: Text Classification
# I pledge my honor that I have abided by the Stevens Honor System

from tqdm.notebook import tqdm
from urllib.parse import urlparse
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from tabulate import tabulate
from datetime import datetime
import numpy as np
import re
import nltk

In [9]:
# Load ham/spam dataset

data = []

with open("a1-data/SMSSpamCollection", "r") as datafile:
    for sentence in datafile.readlines():
        # Get the label and sentence. Store that in the dataset
        split = sentence.split("\t")
        label = split[0]
        sentence = " ".join(split[1:]) 
        data.append([label, sentence])

In [10]:
# Remove punctuation, urls, numbers. Apply lowercase.


# Preprocess the data. Start with detecting any urls. If a url, 
# remove from the sentence. Otherwise apply lowercase and remove
# punctuation and number.
for i in range(len(data)):
    sentence = data[i][1]
    words = sentence.split()
    new_words = []
    for word in words:
        parsed = urlparse(word)
        if parsed.scheme and parsed.netloc:
            continue
        else:
            new_words.append(re.sub(r"[^a-z]", "", word))
    data[i][1] = " ".join(new_words)
    
# Randomize order of data
rand = np.random.default_rng(seed=181853)
data = list(rand.permutation(data))
    

In [11]:
# Tokenization step. Use NLTK
# Get stopwords
try:
    _ = stopwords.words('english')
except LookupError: 
    nltk.download("stopwords")
finally:
    sw = stopwords.words('english')
    
# Create the stemmer
ps = PorterStemmer()    

# Tokenize each sentence, apply stemming, remove stopwords
for i in range(len(data)):
    data[i] = list(data[i])
    sentence = data[i][1]
    # Tokenize
    try:
        _ = word_tokenize(sentence)
    except LookupError:
        nltk.download("punkt")
    finally:
        tokenized = word_tokenize(sentence)
    # Stem + Stopwords
    stemmed = []
    for w in tokenized:
        s = ps.stem(w)
        if s not in sw:
            stemmed.append(s)
    data[i][1] = stemmed


In [12]:
# Split into train, validation, test
# with 
# 70% Train
# 20% validation
# 10% test
# Create distribution table as well
split = len(data) // 10
train = data[:split * 7]
validation = data[split * 7: split * 9]
test = data[split * 9:]

distrib_train = 0
distrib_valid = 0
distrib_test = 0

for elem in train:
    if elem[0] == 'ham':
        distrib_train += 1
for elem in validation:
    if elem[0] == 'ham':
        distrib_valid += 1
for elem in test:
    if elem[0] == 'ham':
        distrib_test += 1

distrib_total = distrib_train + distrib_valid + distrib_test

distribution = [
    ["Training",distrib_train, len(train) - distrib_train, len(train)],
    ["Validation",distrib_valid, len(validation) - distrib_valid, len(validation)],
    ["Testing",distrib_test, len(test) - distrib_test, len(test)],
    ["Total", distrib_total, len(data) - distrib_total, len(data)]
]

print(tabulate(distribution, headers = ["Set", "Ham", "Spam", "Total"]))

Set           Ham    Spam    Total
----------  -----  ------  -------
Training     3394     505     3899
Validation    956     158     1114
Testing       477      84      561
Total        4827     747     5574


In [13]:
# TF-IDF Feature extractor
# Also convert ham/spam class to 0/1

# Build the vocab for TF-IDF
vocab = set()
for _, sentence in train:
    vocab.update(sentence)
    
# Convert the set back to a list (for strict ordering)
vocab = list(vocab)
# Not strictly necessary, but I'm doing it anyway
vocab = sorted(vocab)

# Build document frequency vector

df = [0 for _ in vocab]

for i, word in tqdm(enumerate(vocab), 
                    desc="Building doc freq data",
                    total=len(vocab)):
    for _, doc in train:
        df[i] += doc.count(word)

class TF_IDF:
    # Initialize with vocab, doc frequency
    def __init__(self, vocab, df):
        self.vocab = vocab
        self.df = df
    
    # Converter
    # [tf/idf,...]
    def doc_to_vec(self, doc):
        return np.array([doc.count(word)/df[i] for i, word in enumerate(vocab)])

Building doc freq data:   0%|          | 0/5875 [00:00<?, ?it/s]

In [14]:
# Logistic regression model
class LogisticRegression:
    # Initialization
    # x is np array with x_is as columns
    # y is column of y_is 
    # Randomly initialize w
    # l is the lambda hyperparameter
    # b is the bias vector
    # iter is the current training iteration
    def __init__(self, train_x, train_y, test_x, test_y, valid_x, valid_y, l, descent_method):
        self.train_x = train_x
        self.train_y = train_y
        self.valid_x = valid_x
        self.valid_y = valid_y
        self.test_x = test_x
        self.test_y = test_y
        self.rand = np.random.default_rng(seed=3415)
        self.w = self.rand.normal(size=(1,self.train_x.shape[0]))
        self.l = l
        self.b = self.rand.normal()
        self.iter = 0
        self.descent_method = descent_method
        self.w_grad_sqsum = np.zeros(shape=(1, self.w.shape[1]))
        self.b_grad_sqsum = 0
    
    # Get prediction using sigmoid function
    # Use matrix operations
    def predict(self, x):
        return 1/(1 + np.exp(-(self.w @ x + self.b)))
    
    # Evaluation metrics
    def metrics(self, x, y):
        yhat = np.round(self.predict(x))
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for yi, yhati in zip(y.flatten(), yhat.flatten()):
            if yi == 1 and int(yhati) == 1:
                tp += 1
            elif yi == 1 and int(yhati) == 0:
                fn += 1
            elif yi == 0 and int(yhati) == 1:
                fp += 1
            elif yi == 0 and int(yhati) == 0:
                tn += 1
        try:
            acc = (tp + tn) / (tp + tn + fp + fn)
        except:
            acc = 0
        try:
            prec = tp / (tp + fp)
        except:
            prec = 0
        try:
            rec = tp / (tp + fn)
        except:
            rec = 0
        try:
            f1 = (prec * rec) / (prec + rec)
        except:
            f1 = 0
        return {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1 Score": f1}
                

    # Calculate objective function using matrix operations
    def objective(self, x, y):
        pred = self.predict(x)
        # Avoid errors due to passing 0 to log
        pred = np.clip(pred, 1e-8,1-1e-8)
        logy = np.log(pred)
        log1my = np.log(1 - pred)
        bceloss = -1 / x.shape[1] * np.sum(y * logy + (1 - y) * log1my)
        reg = self.l * np.sum(self.w ** 2)
        return bceloss + reg
    
    # Calculate objective gradient with respect to w and b
    # for specific sample(s) x, y
    def obj_gradient(self, x, y):
        tmp =  np.exp((-(self.w @ x + self.b)))
        denom = 1 + tmp
        w_num = x * (y * tmp - (1 - y))
        s = np.sum(w_num / denom, axis=1, keepdims=True)
        grad_w = -1 / x.shape[1] * s.T + 2 * self.l * self.w
        grad_b = -1 / x.shape[1] * np.sum(y - 1 / denom)
        return grad_w, grad_b
    
    # Adagrad functions
    def w_adagrad(self):
        s = self.w_grad_sqsum / (1 + self.iter)
        s += 1e-8
        return np.sqrt(s)

    def b_adagrad(self):
        s = self.b_grad_sqsum / (1 + self.iter)
        s += 1e-8
        return np.sqrt(s)
    
    # Gradient descent method. Supports other types (stochastic, batch)
    # based on input x, y.
    def apply_grad_descent(self, x, y):
        grad_w, grad_b = self.obj_gradient(x, y)
        self.w_grad_sqsum += grad_w ** 2
        self.b_grad_sqsum += grad_b ** 2
        self.w -= (.5 / np.sqrt(self.iter + 1)) / (self.w_adagrad() if self.descent_method == "vanilla" else 1) * grad_w
        self.b -= (.5 / np.sqrt(self.iter + 1)) / (self.b_adagrad() if self.descent_method == "vanilla" else 1) * grad_b
        
    # grad_descent performs gradient descent based on the set descent method
    # (vanilla, stochastic, minibatch)
    def grad_descent(self):
        if self.descent_method == "vanilla":
            self.apply_grad_descent(self.train_x, self.train_y)
        elif self.descent_method == "stochastic":
            # Permutate the data. the .Ts are there because permutation 
            # randomizes the order along the first axis (rows), but the columns of x
            # are the data.
            data_perm = self.rand.permutation(range(self.train_x.shape[1]))
            for i in data_perm:
                self.apply_grad_descent(np.reshape(self.train_x[:, i], (self.train_x.shape[0], 1)),
                                        np.reshape(self.train_y[0, i], (1, 1)))
            pass
        elif self.descent_method == "minibatch":
            subset = self.rand.choice(range(self.train_x.shape[1]), size=100)
            self.apply_grad_descent(np.reshape(self.train_x[:, subset], (self.train_x.shape[0], 100)),
                                    np.reshape(self.train_y[0, subset], (1,100)))
        else:
            raise ValueError("Undefined gradient descent method. \n"
                             "Expected one of: ('vanilla', 'stochastic', 'minibatch')")
         
    def train(self, max_iter, threshold):
        cur_obj = self.objective(self.valid_x, self.valid_y)
        stop = False
        while self.iter <= max_iter and not stop:
            self.grad_descent()
            new_obj = self.objective(self.valid_x, self.valid_y)
            if np.abs(new_obj - cur_obj) <= threshold:
                print("\n\n")
                print("Stopping early due to reaching improvement threshold.")
                stop = True
            cur_obj = new_obj
            # Delete last iteration output, and print current progress every 100 iterations
            if self.iter % 100 == 0:
                print(f"Iteration {self.iter}: Train Loss={self.objective(self.train_x, self.train_y)}, Validation Loss={new_obj}", end="\r")
            self.iter += 1

In [15]:
# Convert train, validation, test arrays to feature matrices
# Apply standardization
tfidf = TF_IDF(vocab, df)

def convert_to_feature(data_array):
    x = np.array([tfidf.doc_to_vec(doc) for _, doc in data_array]).T
    means = np.mean(x, axis = 1, keepdims=True)
    stdev = np.std(x, axis = 1, keepdims=True)
    stdev[stdev == 0] = 1
    y = np.array([[0 if y == "ham" else 1 for y, _ in data_array]])
    return (x - means) / (stdev), y

train_x, train_y = convert_to_feature(train)
valid_x, valid_y = convert_to_feature(validation)
test_x, test_y = convert_to_feature(test)

In [16]:
# Run training for each type of descent
print("Vanilla gradient descent")
van_start = datetime.now()
LR_van = LogisticRegression(train_x, train_y, test_x, test_y, valid_x, valid_y, .01, "vanilla")
LR_van.train(5000, 1e-15)
van_end = datetime.now()
print("")
print(f"Vanilla gradient descent time = {van_end - van_start} seconds.")
# Because of how I implement vanilla gradient descent (with matrix operations),
# stochastic is far slower than normal (too slow to actually run)
# LR_stoch = LogisticRegression(train_x, train_y, test_x, test_y, valid_x, valid_y, .25, "stochastic")
# print("\n\n")
# LR_stoch.train(5000, 1e-15)
print("Minibatch gradient descent")
mini_start = datetime.now()
LR_mini = LogisticRegression(train_x, train_y, test_x, test_y, valid_x, valid_y, .01, "minibatch")
LR_mini.train(5000, 1e-15)
mini_end = datetime.now()
print("")
print(f"Minibatch gradient descent time = {mini_end - mini_start} seconds.")


LR_mini2 = LogisticRegression(train_x, train_y, test_x, test_y, valid_x, valid_y, .01, "minibatch")
print("")
LR_mini2.train(50000, 1e-15)

Vanilla gradient descent
Iteration 5000: Train Loss=0.03527454029897581, Validation Loss=0.121431840169908661
Vanilla gradient descent time = 0:08:11.465374 seconds.
Minibatch gradient descent
Iteration 5000: Train Loss=2.7575663278694162, Validation Loss=3.1517121746192833
Minibatch gradient descent time = 0:00:30.108415 seconds.



In [17]:
# Print metrics on train, valid, test
def print_metrics(LR):
    train_met = LR.metrics(LR.train_x, LR.train_y)
    valid_met = LR.metrics(LR.valid_x, LR.valid_y)
    test_met = LR.metrics(LR.test_x, LR.test_y)

    metrics = [
        ["Accuracy", train_met["Accuracy"], valid_met["Accuracy"], test_met["Accuracy"]],
        ["Precision", train_met["Precision"], valid_met["Precision"], test_met["Precision"]],
        ["Recall", train_met["Recall"], valid_met["Recall"], test_met["Recall"]],
        ["F1 Score", train_met["F1 Score"], valid_met["F1 Score"], test_met["F1 Score"]]
    ]

    print(tabulate(metrics, headers = ["Metrics", "Training", "Validation", "Testing"]))

print("Vanilla gradient descent:\n")
print_metrics(LR_van)
print("")
print("Minibatch gradient descent:\n")
print_metrics(LR_mini)
print("Longer minibatch gradient descent:\n")
print_metrics(LR_mini)

Vanilla gradient descent:

Metrics      Training    Validation    Testing
---------  ----------  ------------  ---------
Accuracy     0.997948      0.965889   0.953654
Precision    1             1          1
Recall       0.984158      0.759494   0.690476
F1 Score     0.496008      0.431655   0.408451

Minibatch gradient descent:

Metrics      Training    Validation    Testing
---------  ----------  ------------  ---------
Accuracy     0.996409      0.921005   0.942959
Precision    0.991984      0.682292   0.802326
Recall       0.980198      0.829114   0.821429
F1 Score     0.493028      0.374286   0.405882
Longer minibatch gradient descent:

Metrics      Training    Validation    Testing
---------  ----------  ------------  ---------
Accuracy     0.996409      0.921005   0.942959
Precision    0.991984      0.682292   0.802326
Recall       0.980198      0.829114   0.821429
F1 Score     0.493028      0.374286   0.405882


At first, I wasn't have great results. The model kept spitting out around ~0.12 for all inputs,
and I wasn't quite sure why, but then I noticed a couple of errors I made (such as accidentally
multiplying instead of dividing the root mean of the gradients). That didn't fix it, though it did
make the algorithm converge faster.

Then I realized that I hadn't normalized the inputs. Once I did that, things went a lot better. 
The algorithm actually started properly predicting ham/spam.

Besides that, regarding vanilla versus stochastic vs minibatch gradient descent, here's my conclusions

Vanilla: Slow iterations, but converges to a respectable level in only a few hundred

Stochastic: Due to how I implemented vanilla with matrix operations, and how stochastic is a loop by design,
stochastic ended up being way too slow per loop over the data to be run in a feasible amount of time.

Minibatch: Much faster iterations (about 5000 in the time it takes vanilla to run 300), but converges slower 
than vanilla (in strictly iteration terms). However, due to the vastly faster iteration time, the algorithm 
converges to a similar level as vanilla in shorter clock time.

In only a few minutes of training, the metrics for this model on the training data are nearly 100%. 
However, validation and train metrics are lower. Validation error did continuously decrease over time, 
but this is either a symptom of overfitting or the training data not being a complete representation of
the overall data. 

I did notice, when regularizing the data, that the standard deviation would be zero in some cases (
meaning that some words were not used in any of the training documents), so that might contribute. 

In [18]:
try:
    del LR_van
    del LR_mini
    del LR_mini2
except:
    pass
# Set up k-fold cross validation to find best lambda (with k = 10)
# Maintain 10% of the data as testing data
# Use other 90% in k-fold

# Use minibatch
split = len(data) // 11

test = data[:split]
sections = []
for i in range(1,11):
    sections.append((split*i, split*(i+1)))

test_x, test_y = convert_to_feature(test)
models = {}
lambdas = [0.01,0.02,0.05,0.1,0.2,0.5]

for k in range(10):
    first, second = sections[k]
    validation = data[first:second]
    train = data[split:first]
    train2 = data[second:]
    train.extend(train2)
    train_x, train_y = convert_to_feature(train)
    valid_x, valid_y = convert_to_feature(validation)
    for l in lambdas:
        LR = LogisticRegression(train_x, train_y, test_x, test_y, valid_x, valid_y, l, "minibatch")
        LR.train(2500, 1e-15)
        models[(k, l)] = LR.objective(LR.train_x, LR.train_y)
        del LR

In [19]:
# Which is the best l?
min_avg = None

for l in lambdas:
    acc = 0
    for k in range(10):
        acc += models[(k, l)]
    acc /= 10
    if (min_avg is None) or acc < min_avg[1]:
        min_avg = (l, acc)
print(f"Best lambda = {min_avg[0]}, with an average validation loss of {min_avg[1]} after 2500 iterations.")

Best lambda = 0.05, with an average validation loss of 0.1015926515861364 after 2500 iterations.


In [20]:
# Load book dataset and perform all preprocessing
# then print the data distribution table

bookdata = []

with open("a1-data/books.txt", "r", encoding="utf8") as datafile:
    for line in datafile.readlines():
        # Get the label and sentence. Store that in the dataset
        split = line.split("\t")
        author = split[0]
        line = " ".join(split[1:]) 
        bookdata.append([author, line])
        
# Remove punctuation, urls, numbers. Apply lowercase.

# Preprocess the data. Start with detecting any urls. If a url, 
# remove from the sentence. Otherwise apply lowercase and remove
# punctuation and number.
for i in range(len(bookdata)):
    line = bookdata[i][1]
    words = line.split()
    new_words = []
    for word in words:
        parsed = urlparse(word)
        if parsed.scheme and parsed.netloc:
            continue
        else:
            new_words.append(re.sub(r"[^a-z]", "", word))
    bookdata[i][1] = " ".join(new_words)
    
# Randomize order of data
rand = np.random.default_rng(seed = 31135)
bookdata = list(rand.permutation(bookdata))

# Tokenization step. Use NLTK
# Get stopwords
try:
    _ = stopwords.words('english')
except LookupError: 
    nltk.download("stopwords")
finally:
    sw = stopwords.words('english')
    
# Create the stemmer
ps = PorterStemmer()    

# Tokenize each sentence, apply stemming, remove stopwords
for i in range(len(bookdata)):
    bookdata[i] = list(bookdata[i])
    line = bookdata[i][1]
    # Tokenize
    try:
        _ = word_tokenize(line)
    except LookupError:
        nltk.download("punkt")
    finally:
        tokenized = word_tokenize(line)
    # Stem + Stopwords
    stemmed = []
    for w in tokenized:
        s = ps.stem(w)
        if s not in sw:
            stemmed.append(s)
    bookdata[i][1] = stemmed

# Split into train, validation, test
# with 
# 70% Train
# 20% validation
# 10% test
# Create distribution table as well
split = len(bookdata) // 10
train = bookdata[:split * 7]
validation = bookdata[split * 7: split * 9]
test = bookdata[split * 9:]

authors = ["Jane Austen", "Arthur Conan Doyle", "Fyodor Dostoyevsky"]

distrib = {dset: {author: 0 for author in authors} for dset in ["train", "valid", "test"]}

for elem in train:
    distrib["train"][elem[0]] += 1
for elem in validation:
    distrib["valid"][elem[0]] += 1
for elem in test:
    distrib["test"][elem[0]] += 1

distribution = [
    ["Training", 
     distrib["train"]["Jane Austen"],
     distrib["train"]["Arthur Conan Doyle"],
     distrib["train"]["Fyodor Dostoyevsky"],
     distrib["train"]["Jane Austen"]+distrib["train"]["Arthur Conan Doyle"]+distrib["train"]["Fyodor Dostoyevsky"]],
    ["Validation",distrib["valid"]["Jane Austen"],
     distrib["valid"]["Arthur Conan Doyle"],
     distrib["valid"]["Fyodor Dostoyevsky"],
     distrib["valid"]["Jane Austen"]+distrib["valid"]["Arthur Conan Doyle"]+distrib["valid"]["Fyodor Dostoyevsky"]],
    ["Testing",distrib["test"]["Jane Austen"],
     distrib["test"]["Arthur Conan Doyle"],
     distrib["test"]["Fyodor Dostoyevsky"],
     distrib["test"]["Jane Austen"]+distrib["test"]["Arthur Conan Doyle"]+distrib["test"]["Fyodor Dostoyevsky"]],
    ["Total", distrib["train"]["Jane Austen"] + distrib["valid"]["Jane Austen"] + distrib["test"]["Jane Austen"],
     distrib["train"]["Arthur Conan Doyle"] + distrib["valid"]["Arthur Conan Doyle"] + distrib["test"]["Arthur Conan Doyle"],
     distrib["train"]["Fyodor Dostoyevsky"] + distrib["valid"]["Fyodor Dostoyevsky"] + distrib["test"]["Fyodor Dostoyevsky"],
     len(bookdata)]
]

print(tabulate(distribution, headers = ["Set", "Jane Austen", "Arthur Conan Doyle", "Fyodor Dostoyevsky", "Total"]))

Set           Jane Austen    Arthur Conan Doyle    Fyodor Dostoyevsky    Total
----------  -------------  --------------------  --------------------  -------
Training             7757                  1759                  4155    13671
Validation           2193                   525                  1188     3906
Testing              1104                   254                   601     1959
Total               11054                  2538                  5944    19536


In [25]:
# Set up TF_IDF for bookdata
# Build the vocab for TF-IDF
vocab = set()
for _, sentence in train:
    vocab.update(sentence)
    
# Convert the set back to a list (for strict ordering)
vocab = list(vocab)
# Not strictly necessary, but I'm doing it anyway
vocab = sorted(vocab)

# Build document frequency vector
df = [0 for _ in vocab]

for i, word in tqdm(enumerate(vocab), 
                    desc="Building doc freq data",
                    total=len(vocab)):
    for _, doc in train:
        df[i] += doc.count(word)

# Convert bookdata to data matrix
tfidf = TF_IDF(vocab, df)

def convert_to_feature(data_array):
    x = np.array([tfidf.doc_to_vec(doc) for _, doc in tqdm(data_array)]).T
    means = np.mean(x, axis = 1, keepdims=True)
    stdev = np.std(x, axis = 1, keepdims=True)
    stdev[stdev == 0] = 1
    y = np.array([[1 if i == authors.index(y) else 0 for i in range(3)] 
                   for y, _ in tqdm(data_array)]).T
    return (x - means) / (stdev), y

train_x, train_y = convert_to_feature(train)
valid_x, valid_y = convert_to_feature(validation)
test_x, test_y = convert_to_feature(test)

Building doc freq data:   0%|          | 0/17015 [00:00<?, ?it/s]

  0%|          | 0/13671 [00:00<?, ?it/s]

  0%|          | 0/13671 [00:00<?, ?it/s]

  0%|          | 0/3906 [00:00<?, ?it/s]

  0%|          | 0/3906 [00:00<?, ?it/s]

  0%|          | 0/1959 [00:00<?, ?it/s]

  0%|          | 0/1959 [00:00<?, ?it/s]

In [ ]:
# Multiclass logistic regression model
class MultiClassLogisticRegression:
    # Initialization
    # x is np array with x_is as columns
    # y is column of y_is 
    # Randomly initialize w
    # l is the lambda hyperparameter
    # b is the bias vector
    # iter is the current training iteration
    def __init__(self, train_x, train_y, test_x, test_y, valid_x, valid_y, l, descent_method):
        self.train_x = train_x
        self.train_y = train_y
        self.valid_x = valid_x
        self.valid_y = valid_y
        self.test_x = test_x
        self.test_y = test_y
        self.rand = np.random.default_rng(seed=3415)
        self.w = self.rand.normal(scale=0.1,size=(train_y.shape[0], self.train_x.shape[0]))
        self.l = l
        self.iter = 0
        self.descent_method = descent_method
        self.w_grad_sqsum = np.zeros(shape=(train_y.shape[0], self.w.shape[1]))
    
    # Get prediction using sigmoid function
    # Use matrix operations
    def predict(self, x):
        tmp = np.exp(self.w @ x)
        return tmp / np.sum(tmp, axis=0, keepdims=True)
    
    # Evaluation metrics
    def metrics(self, x, y):
        yhat = self.predict(x)
        table = np.zeros(size=(3,3))
        for yi, yhati in zip(y.T, yhat.T):
            table[np.argmax(yi), np.argmax(yhati)] += 1
        return table

    # Calculate objective function using matrix operations
    def objective(self, x, y):
        pred = self.predict(x)
        # Avoid errors due to passing 0 to log
        pred = np.clip(pred, 1e-8,1-1e-8)
        logp = np.log(pred)
        bceloss = -1 / x.shape[1] * np.sum(y * logp)
        return bceloss
    
    # Calculate objective gradient with respect to w and b
    # for specific sample(s) x, y
    def obj_gradient(self, x, y):
        tmp = self.predict(x)
        err = y - tmp
        grad = np.zeros_like(self.w)
        for i in range(x.shape[1]):
            grad -= err[:,[i]] @ x[:,[i]].T
        return grad
    
    # Adagrad functions
    def w_adagrad(self):
        s = self.w_grad_sqsum / (1 + self.iter)
        s += 1e-8
        return np.sqrt(s)
    
    # Gradient descent method. Supports other types (stochastic, batch)
    # based on input x, y.
    def apply_grad_descent(self, x, y):
        grad_w = self.obj_gradient(x, y)
        self.w_grad_sqsum += grad_w ** 2
        self.w -= (.05 / np.sqrt(self.iter + 1)) / (self.w_adagrad() if self.descent_method == "vanilla" else 1) * grad_w
        
    # grad_descent performs gradient descent based on the set descent method
    # (vanilla, stochastic, minibatch)
    def grad_descent(self):
        if self.descent_method == "vanilla":
            self.apply_grad_descent(self.train_x, self.train_y)
        elif self.descent_method == "stochastic":
            # Permutate the data. the .Ts are there because permutation 
            # randomizes the order along the first axis (rows), but the columns of x
            # are the data.
            data_perm = self.rand.permutation(range(self.train_x.shape[1]))
            for i in data_perm:
                self.apply_grad_descent(np.reshape(self.train_x[:, i], (self.train_x.shape[0], 1)),
                                        np.reshape(self.train_y[0, i], (1, 1)))
            pass
        elif self.descent_method == "minibatch":
            subset = self.rand.choice(range(self.train_x.shape[1]), size=100)
            self.apply_grad_descent(np.reshape(self.train_x[:, subset], (self.train_x.shape[0], 100)),
                                    np.reshape(self.train_y[0, subset], (1,100)))
        else:
            raise ValueError("Undefined gradient descent method. \n"
                             "Expected one of: ('vanilla', 'stochastic', 'minibatch')")
         
    def train(self, max_iter, threshold):
        cur_obj = self.objective(self.valid_x, self.valid_y)
        stop = False
        while self.iter <= max_iter and not stop:
            self.grad_descent()
            new_obj = self.objective(self.valid_x, self.valid_y)
            if np.abs(new_obj - cur_obj) <= threshold:
                print("\n\n")
                print("Stopping early due to reaching improvement threshold.")
                stop = True
            cur_obj = new_obj
            # Delete last iteration output, and print current progress every 100 iterations
            if self.iter % 100 == 0:
                print(f"Iteration {self.iter}: Train Loss={self.objective(self.train_x, self.train_y)}, Validation Loss={new_obj}", end="\r")
            self.iter += 1

In [ ]:
MCLR = MultiClassLogisticRegression(train_x, train_y, test_x, test_y, valid_x, valid_y, 0.05, "minibatch")
MCLR.train(50000, 1e-15)

69.5865601057272
62.29316181032853
528.5462678791644
6064.937429127756


C:\Users\Aidan\AppData\Local\Temp\ipykernel_11952\3116882647.py:28: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(self.w @ x)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_11952\3116882647.py:29: RuntimeWarning: invalid value encountered in divide
  return tmp / np.sum(tmp, axis=0, keepdims=True)


334.026126418091in Loss=nan, Validation Loss=7.56059585330592
1079.0509616491538
243.73724132657986
1145.872358495514
8988.406554534997
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nanration 100: Train Loss=nan, Validation Loss=nan
nan
nan
nan
nan
nan
nan
nan
n

Prediction kept giving overflow errors. Pretty sure I have the right gradient, but it could be wrong. 